In [1]:
import cplex
import networkx as nx

In [229]:
def read_dimacs_graph(file_path):
    '''
        Parse .col file and return graph object
    '''
    edges = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('c'):  # graph description
                print(*line.split()[1:])
            # first line: p name num_of_vertices num_of_edges
            elif line.startswith('p'):
                _, name, vertices_num, edges_num = line.split()
                print('{0} {1} {2}'.format(name, vertices_num, edges_num))
            elif line.startswith('e'):
                _, v1, v2 = line.split()
                edges.append((v1, v2))
            else:
                continue
        return nx.Graph(edges)

In [231]:
solve_integer = True
if solve_integer:
    ONE = 1
    ZERO = 0
    print("Solve ILP")
else:
    ONE = 1.0
    ZERO = 0.0
    print("Solve LP")

Solve ILP


In [232]:
graph = read_dimacs_graph("benchs/max_clique_txt/DIMACS_all_ascii/san200_0.7_2.clq")
not_connected_edges_list = list(nx.complement(graph).edges)

File: san200_0.7_2.clq

SOURCE: Laura Sanchis (laura@cs.colgate.edu)

REFERENCE: "Test Case Construction for the Vertex Cover Problem",
DIMACS Workshop on Computational Support for Discrete Mathematics,
March, 1992, with additional work in a to be published technical
report.

200 vertices 13930 edges 18 max clique
200 5970 182 3352 291624 12
edge 200 13930


In [233]:
list_nodes = list(graph.nodes)
list_nodes_int = [int(i) for i in list_nodes]
list_nodes_int.sort()

In [234]:
names = ['x' + str(i) for i in list_nodes_int]
objective = [ONE] * max(list_nodes_int)
lower_bounds = [ZERO] * max(list_nodes_int)
upper_bounds = [ONE] * max(list_nodes_int)

In [235]:
problem = cplex.Cplex()
problem.objective.set_sense(problem.objective.sense.maximize)
problem.variables.add(obj = objective,
                      lb = lower_bounds,
                      ub = upper_bounds,
                      names = names)

range(0, 200)

In [236]:
constraint_names = ["c" + str(i) for i in range(len(not_connected_edges_list))]

constraints = [[['x' + edges_pair[0], 'x' + edges_pair[1]], [ONE, ONE]] for edges_pair in not_connected_edges_list]
rhs = [ONE] * len(constraints)
constraint_senses = ["L"] * len(constraints)

In [237]:
problem.linear_constraints.add(lin_expr = constraints,
                               senses = constraint_senses,
                               rhs = rhs,
                               names = constraint_names)
for i in list_nodes_int:
    if solve_integer:
        problem.variables.set_types(i - 1, problem.variables.type.binary)
    else:
        problem.variables.set_types(i - 1, problem.variables.type.continuous)

problem.solve()
val = problem.solution.get_values()

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.04 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 4966 rows and 0 columns.
MIP Presolve modified 1004 coefficients.
Reduced MIP has 1004 rows, 200 columns, and 7521 nonzeros.
Reduced MIP has 200 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (17.99 ticks)
Probing time = 0.00 sec. (0.83 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 1004 rows, 200 columns, and 7521 nonzeros.
Reduced MIP has 200 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (7.22 ticks)
Probing time = 0.00 sec. (0.83 ticks)
Clique table members: 1004.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.08 sec. (68.56 ticks)

        Nodes                       

In [238]:
problem.solution.get_objective_value()

18.0

In [239]:
import pprint
pprint.pprint(problem.variables.get_num_semicontinuous.__doc__)

('Returns the number of semi-continuous variables in the problem.\n'
 '\n'
 '        Example usage:\n'
 '\n'
 '        >>> import cplex\n'
 '        >>> c = cplex.Cplex()\n'
 '        >>> t = c.variables.type\n'
 '        >>> indices = c.variables.add(types = [t.semi_continuous, '
 't.semi_integer, t.semi_integer])\n'
 '        >>> c.variables.get_num_semicontinuous()\n'
 '        1\n'
 '        ')
